In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/sample_submission.csv
/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/train.csv
/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/test.csv


Predict the cost to ship the sculptures
It can be difficult to navigate the logistics when it comes to buying art. These include, but are not limited to, the following:

Effective collection management
Shipping the paintings, antiques, sculptures, and other collectibles to their respective destinations after purchase
Though many companies have made shipping consumer goods a relatively quick and painless procedure, the same rules do not always apply while shipping paintings or transporting antiques and collectibles.

**We will predict the cost required to ship these sculptures to customers based on the information provided in the dataset.**

**Customer Id**
Represents the unique identification number of the customers

**Artist Name**  
Represents the name of the artist

**Artist Reputation**    
Represents the reputation of an artist in the market (the greater the reputation value, the higher the reputation of the artist in the market)

**Height**
Represents the height of the sculpture

**Width**
Represents the width of the sculpture

**Weight**
Represents the  weight of the sculpture

**Material**
Represents the material that the sculpture is made of

**Price Of Sculpture**
Represents the price of the sculpture

**Base Shipping Price**
Represents the base price for shipping a sculpture

International
Represents whether the shipping is international

Express Shipment
Represents whether the shipping was in the express (fast) mode

**Installation Included**
Represents whether the order had installation included in the purchase of the sculpture

**Transport**
Represents the mode of transport of the order

**Fragile**
Represents whether the order is fragile

**Customer Information**
Represents  details about a customer

**Remote Location**
Represents whether the customer resides in a remote location

**Scheduled Date**
Represents the date when the order was placed

**Delivery Date**
Represents the date of delivery of the order

**Customer Location**
Represents the location of the customer


***Target Feature***

**Cost**
Represents the cost of the order

In [4]:
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Reading the dataset

In [5]:
df = pd.read_csv('/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/train.csv')
df_test = pd.read_csv('/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/test.csv')

In [6]:
df.head(5)

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777",-283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072",-159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241",-154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375",-161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793",-159.23


Target feature values are negative, getting absolute values

In [7]:
df['Cost'] = df['Cost'].abs()

Let's get some basic understanding about our data, with pandas profiling

In [8]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, title="Your Dataset", explorative=True)
# profile.to_notebook_iframe()

Some important conclusions:

1. The dataset contains many outliers, in Height,Width,Weight, Artist Reputation, Remote Location, Transport and Material which needs to be removed.
2. Data has many missing values which are to be handled.
3. There is high correlation between height, width, weight, Price of Sculpture, base cost.

Getting more details about missing values

In [9]:
def summary(df):
    print(f"Dataset Shape: {df.shape}")
    summary= pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary= summary.reset_index()
    summary['Feature Name'] = summary['index']
    summary = summary[['Feature Name', 'dtypes']]
    summary['missing'] = df.isnull().sum().values
    summary['Uniques'] = df.nunique().values
    return summary

summary(df)

Dataset Shape: (6500, 20)


,Feature Name,dtypes,missing,Uniques
0,Customer Id,object,0,6500
1,Artist Name,object,0,6449
2,Artist Reputation,float64,750,101
3,Height,float64,375,65
4,Width,float64,584,40
5,Weight,float64,587,4410
6,Material,object,764,7
7,Price Of Sculpture,float64,0,3424
8,Base Shipping Price,float64,0,3732
9,International,object,0,2


Inferences:

* Customer ID is unique for each customer, its not important in model training so it should be dropped.

* Scheduled Date, Delivery Date and Customer Location needs to be converted into date datatype

* Name of Artist is also not of much importance so it should be dropped.

In [10]:
df.drop(['Customer Id','Artist Name'],axis =1, inplace = True)

In [11]:
df['Scheduled Date'] = pd.to_datetime(df['Scheduled Date'])
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'])
df['duration'] = (df['Scheduled Date']- df['Delivery Date']).dt.days
df['duration'].value_counts(sort = True)

 0    1374
 4     676
-1     669
 2     660
-2     635
 1     634
-4     628
 3     623
-3     601
Name: duration, dtype: int64

Since half of the values are negative, i.e., scheduled date is more than delivery date(which is not possible). So we will drop scheduled date, delivery date and duration feature.

In [12]:
df.drop(['Scheduled Date','Delivery Date','duration'], axis = 1, inplace = True)

Feature Creation

Creating State column from Customer Location

In [13]:
df['State']= df['Customer Location'].str.split(" ").str[-2]
df.drop(['Customer Location'], axis = 1, inplace = True)
df['State'].sample(3)

2688    AK
4788    NM
3236    NJ
Name: State, dtype: object

# Handling Missing Values

We will import the missing values of 'Height','Width', 'Artist Reputation', 'Remote Location','Transport','Material' randomly, by using below function.

In [14]:
def random_imputation(df, cols):
    for col in cols:
        imputed_col_values = np.random.choice(df[~df[col].isna()][col].values, size = df[col].isna().sum())
        col_null_indices = df[df[col].isna()].index
        df.loc[col_null_indices, col] = imputed_col_values
    return df

This function first selects random values from non missing values, and than impute them in missing value indices with those random values.

In [15]:
df = random_imputation(df , ['Height','Width', 'Artist Reputation', 'Remote Location','Transport','Material'])

In [16]:
summary(df)

Dataset Shape: (6500, 16)


,Feature Name,dtypes,missing,Uniques
0,Artist Reputation,float64,0,101
1,Height,float64,0,65
2,Width,float64,0,40
3,Weight,float64,587,4410
4,Material,object,0,7
5,Price Of Sculpture,float64,0,3424
6,Base Shipping Price,float64,0,3732
7,International,object,0,2
8,Express Shipment,object,0,2
9,Installation Included,object,0,2


In [17]:
df.drop('State', axis =1, inplace = True)

Since weight is highly correlated with height, weight, Price, Material, we will impute the missing values of weight using knn imputer.

But before using knn, we will perform one hot encoding, for the categorical features.

In [18]:
cat_cols = [col for col in df.columns if df[col].dtype == 'object']
num_cols = [col for col in df.columns if col not in cat_cols]

In [19]:
encoder = OneHotEncoder(sparse_output= False, handle_unknown = 'ignore')
df_encoded = encoder.fit_transform(df[cat_cols])
df_encoded = pd.DataFrame(df_encoded,  columns = encoder.get_feature_names_out(cat_cols))
            

Performing KNN Imputation on dataset to impute values of weight column

In [20]:
num_cols

['Artist Reputation',
 'Height',
 'Width',
 'Weight',
 'Price Of Sculpture',
 'Base Shipping Price',
 'Cost']

In [21]:
num_cols.remove('Cost')
X = pd.concat([df[num_cols] ,df_encoded], axis= 1)

In [22]:
imputer = KNNImputer(n_neighbors=5)
imputed_df = imputer.fit_transform(X)
X = pd.DataFrame(imputed_df, columns=X.columns)
df_new = pd.concat([X , df['Cost']], axis =1)

# Removing Outliers

In [23]:
def remove_outliers_iqr(dataframe, columns):
    for col in columns:
        data = dataframe[col]
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        cleaned_dataframe = dataframe[(data > lower_bound) & (data < upper_bound)]
    return cleaned_dataframe

In [24]:
df_new = remove_outliers_iqr(df_new, ['Height','Width','Weight','Price Of Sculpture'])

In [25]:
df_new.shape

(5312, 29)

**Splitting features  X and y**

In [39]:
X = df_new.drop('Cost', axis = 1)
y = df_new['Cost']

**Train-Test Split**

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)        

Using Lazy predict to select best model

In [28]:
# !pip install lazypredict

In [29]:
# from lazypredict.Supervised import LazyRegressor
# reg = LazyRegressor(ignore_warnings=False, custom_metric=None)
# models, predictions = reg.fit(X_train, X_test, y_train, y_test)

Training Model with XGBRegressor

In [ ]:
xgb = XGBRegressor()


param_grid = {
        'n_estimators': [50,100,200],
        'learning_rate': [0.1,0.2],
        'min_child_weight': [1,2],
        'subsample': [0.6,0.8],
        'colsample_bytree': [0.6, 0.8],
        'max_depth': [5]
        }

xgb_grid = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5)
xgb_grid.fit(X_train, y_train)

y_pred = xgb_grid.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('Best parameters:', xgb_grid.best_params_)
print('RMSE:', rmse)
print('R-squared:', r2)